In [6]:
# import packages
import numpy as np

In [9]:
# constants
LEFT, RIGHT = 0, 1
START, END = 0, 1

In [ ]:
# environment implementation
class Environment:
    def __init__(self):
        self.reset()

    def reset(self):
        self.state = START
        return self.state

    def step(self, action):
        if action == RIGHT:
            reward = 0.0
            self.state = END
        else:
            if np.random.rand() < 0.9:
                reward = 0.0
                self.state = START
            else:
                reward = 1.0
                self.state = END
        return reward, self.state

In [ ]:
# agent implementation
class Agent:
    def __init__(self, IS='OIS'):
        self.IS = IS
        self.rewards = []
        self.rho = 0
        self.t = 0

    def act(self):
        action = LEFT if np.random.rand() < 0.5 else RIGHT
        self.t += 1
        return action
    
    def learn(self, state, action, reward, next_state):
        self.rho = 0 if action == RIGHT else 1.0 / (0.5 ** self.t)
        self.rho = 1.0 / pow(0.5, len(trajectory))

In [ ]:
for trial in np.arange(1e3):
    agent = Agent()
    env = Environment()
    state = env.reset()
    done = False
    for t in np.arange(1e4):
        if done:
            state = env.reset()
        action = agent.act(state)
        reward, next_state, done = env.step(action)
        agent.learn(state, reward, next_state, done)